In [ ]:
import cv2
import numpy as np
import pyfirmata
import threading
import time
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from filterpy.kalman import KalmanFilter

# --- Kết nối Arduino ---
port = "COM3"
board = pyfirmata.Arduino(port)
servo_pinX = board.get_pin('d:9:s')   # Servo trục X
servo_pinY = board.get_pin('d:10:s')  # Servo trục Y

# --- Load YOLO ---
model = YOLO("model.pt")

# --- DeepSORT ---
tracker = DeepSort(max_age=150, max_iou_distance=0.4, nn_budget=100)

# --- Kalman Filter ---
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])
kf.P *= 1000
kf.x = np.array([0, 0, 0, 0])

# --- Camera ---
ws, hs = 1280, 720
cap = cv2.VideoCapture(1)
cap.set(3, ws)
cap.set(4, hs)

# --- Theo dõi ---
selected_id = None
detections = []
latest_frame = None
yolo_lock = threading.Lock()
lost_tracks = defaultdict(lambda: {"last_seen": 0, "track": None})
LOST_TIMEOUT = 5  # giây

# --- Khởi tạo servo ---
current_servo_x = 90
current_servo_y = 90
servo_pinX.write(current_servo_x)
servo_pinY.write(current_servo_y)
threshold_error = 10
step = 1.2

# --- Hàm YOLO chạy song song ---
def run_yolo_loop():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame_copy = latest_frame.copy()
            results = model(frame_copy)
            new_detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0].item()
                class_id = int(box.cls[0].item())
                if class_id == 0:  # Người
                    new_detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))
            detections = new_detections

threading.Thread(target=run_yolo_loop, daemon=True).start()

# --- Mouse callback chọn người ---
cv2.namedWindow("Tracking")
def select_object(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for track in param:
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_id = track.track_id
                print(f"🔍 Chọn đối tượng ID: {selected_id}")
cv2.setMouseCallback("Tracking", select_object, param=[])

# --- Vòng lặp chính ---
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    current_time = time.time()

    tracks = tracker.update_tracks(detections, frame=frame)
    cv2.setMouseCallback("Tracking", select_object, param=tracks)

    active_ids = set()

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        active_ids.add(track_id)

        if track_id in lost_tracks:
            del lost_tracks[track_id]

        # --- Theo dõi người được chọn ---
        if selected_id is not None and track_id == selected_id:
            kf.predict()
            kf.update([cx, cy])
            predicted = kf.x[:2]
            px, py = predicted[0], predicted[1]

            # Tính sai số tâm khung hình
            center_x = ws // 2
            center_y = hs // 2
            error_x = center_x - px
            error_y = center_y - py

            # Điều chỉnh servo nếu lệch quá ngưỡng
            if abs(error_x) > threshold_error:
                current_servo_x += step if error_x > 0 else -step
            if abs(error_y) > threshold_error:
                current_servo_y += step if error_y > 0 else -step

            # Giới hạn servo trong 0–180 độ
            current_servo_x = np.clip(current_servo_x, 0, 180)
            current_servo_y = np.clip(current_servo_y, 0, 180)

            # Gửi lệnh servo
            servo_pinX.write(current_servo_x)
            servo_pinY.write(current_servo_y)

        # --- Vẽ bounding box ---
        color = (0, 255, 0) if track_id == selected_id else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID {track_id}"
        if selected_id == track_id:
            label += f" X:{current_servo_x:.0f} Y:{current_servo_y:.0f}"
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # --- Xử lý mất dấu ---
    for track in tracker.tracker.tracks:
        track_id = track.track_id
        if track_id not in active_ids and track_id == selected_id:
            if track_id not in lost_tracks:
                lost_tracks[track_id] = {"last_seen": current_time, "track": track}
            elif current_time - lost_tracks[track_id]["last_seen"] > LOST_TIMEOUT:
                print(f"❌ Mất dấu ID {track_id}, hủy theo dõi sau 5s.")
                selected_id = None
                del lost_tracks[track_id]

    # --- Vẽ ghost box ---
    for track_id, data in lost_tracks.items():
        if current_time - data["last_seen"] <= LOST_TIMEOUT:
            track = data["track"]
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)
            cv2.putText(frame, f"ID {track_id} (ghost)", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # --- Hiển thị ---
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



General configuration for OpenCV 4.6.0 =====================================
  Version control:               4.6.0

  Platform:
    Timestamp:                   2022-06-07T10:17:42Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.22.5
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (4 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (31 files):           + SSSE3 SSE4_1 

In [3]:
import cv2

for i in range(5):  # thử từ 0 đến 4
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera index {i} đang hoạt động")
        cap.release()
    else:
        print(f"Camera index {i} không hoạt động")


Camera index 0 đang hoạt động
Camera index 1 không hoạt động
Camera index 2 không hoạt động
Camera index 3 không hoạt động
Camera index 4 không hoạt động


NameError: name 'cv2' is not defined